In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
input_size = 3
batch_size = 5
eps = 1e-1

In [3]:
class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum 
        self.list1 = []
        self.running_var = 1
        self.running_mean = 0
        self.flag = True # Реализуйте в этом месте конструктор.

    def __call__(self, input_tensor):
      #print(input_tensor)
      ten_sor = input_tensor.numpy()
      if self.flag == True:
        matrix_mean = np.mean(ten_sor, axis = 0)
        math_t = (np.mean((ten_sor - matrix_mean)**2, axis = 0))
        ten_sor = (ten_sor -  matrix_mean)/np.sqrt(math_t + self.eps)
        
        self.running_mean = (1 - self.momentum) * matrix_mean + self.running_mean * self.momentum
        self.running_var = (1 - self.momentum) * math_t * batch_size / (batch_size-1) + self.momentum * self.running_var
      else:   
        ten_sor = (ten_sor -  self.running_mean)/np.sqrt(self.running_var + self.eps)
  
      normed_tensor = torch.from_numpy(ten_sor) * self.weight + self.bias # Напишите в этом месте нормирование входного тензора.
      return normed_tensor

    def eval(self):
      self.flag = False # В этом методе реализуйте переключение в режим предикта.


In [4]:

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)


In [5]:
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
         and norm_output.shape == custom_output.shape
    
print(all_correct)
batch_norm.eval()
custom_batch_norm1d.eval()
print(batch_norm._buffers)

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output) \
        and norm_output.shape == custom_output.shape
    #print('1', norm_output,'2', custom_output)
print(all_correct)

True
OrderedDict([('running_mean', tensor([ 0.1603, -0.2099,  0.2156])), ('running_var', tensor([1.2720, 0.4545, 0.5378])), ('num_batches_tracked', tensor(8))])
[ 0.1602912  -0.20985311  0.2156185 ] & [1.2719766  0.45447376 0.5377768 ]
[ 0.1602912  -0.20985311  0.2156185 ] & [1.2719766  0.45447376 0.5377768 ]
[ 0.1602912  -0.20985311  0.2156185 ] & [1.2719766  0.45447376 0.5377768 ]
[ 0.1602912  -0.20985311  0.2156185 ] & [1.2719766  0.45447376 0.5377768 ]
[ 0.1602912  -0.20985311  0.2156185 ] & [1.2719766  0.45447376 0.5377768 ]
[ 0.1602912  -0.20985311  0.2156185 ] & [1.2719766  0.45447376 0.5377768 ]
[ 0.1602912  -0.20985311  0.2156185 ] & [1.2719766  0.45447376 0.5377768 ]
[ 0.1602912  -0.20985311  0.2156185 ] & [1.2719766  0.45447376 0.5377768 ]
True
